# Try out CNN on averaged EEG data

## Pre-processing
+ Import data.
+ Apply filters (bandpass).
+ Detect potential bad channels and replace them by interpolation.
+ Detect potential bad epochs and remove them.
+ Average over a number of randomly drawn epochs (of same person and same stimuli).

## Train CNN network
+ Define network architecture
+ Split data
+ Train model


## Import packages & links

In [1]:
# Import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mne
#%matplotlib inline

from mayavi import mlab

In [2]:
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_ePodium\\"
PATH_CODE = ROOT + "EEG_explorer\\"
PATH_DATA = ROOT + "Data\\"
PATH_OUTPUT = ROOT + "Data\\processed\\"
PATH_METADATA = ROOT + "Data\\metadata\\"
PATH_MODELS = ROOT + "trained_models\\"
file_labels = "metadata.xlsx"

import sys
sys.path.insert(0, PATH_CODE)

## Load pre-processed dataset
+ See notebook for preprocessing: Exploring_EEG_data_04_prepare_data_for_ML.ipynb

In [3]:
filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected.npy"
signal_collection = np.load(filename)

filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected_labels.npy"
label_collection = np.load(filename)

In [4]:
import csv
metadata = []
filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected_metadata.csv"
with open(filename, 'r') as readFile:
    reader = csv.reader(readFile, delimiter=',')
    for row in reader:
        if len(row) > 0:
            metadata.append(row)
readFile.close()

In [5]:
metadata[:10]

[['1', '034_17_mc_mmn36_wk.cnt', '443'],
 ['2', '036_17_mc_mmn36_wk.cnt', '1384'],
 ['18', '175_17_jd_mmn_wk.cnt', '1858'],
 ['26', '305_17_jc_mmn36_wk.cnt', '2349'],
 ['27', '306_17_mc_mmn36_wk.cnt', '2813'],
 ['28', '307_17_jc_mmn36_wakker.cnt', '3447'],
 ['29', '308_17_jc_mmn36_wk.cnt', '3938'],
 ['30', '309_17_jc_mmn.cnt', '4425'],
 ['33', '314_17_mc_mmn36_wk.cnt', '4917'],
 ['38', '337_17_jc_mmn36_wk.cnt', '5389']]

## Create training data --> Scale/normalize data

In [6]:
min_maxes = []
for ID in range(1,len(metadata)):
    min_maxes.append((signal_collection[int(metadata[ID-1][2]):int(metadata[ID][2]),:,:].min(), 
                     signal_collection[int(metadata[ID-1][2]):int(metadata[ID][2]),:,:].max(), 
                     signal_collection[int(metadata[ID-1][2]):int(metadata[ID][2]),:,:].mean(), 
                     signal_collection[int(metadata[ID-1][2]):int(metadata[ID][2]),:,:].var()))

In [7]:
data_max = np.mean([x[1] for x in min_maxes]) #signal_collection.max()
data_min = np.mean([x[0] for x in min_maxes]) #signal_collection.min()
data_mean = signal_collection.mean()

signal_collection = signal_collection - data_mean
signal_collection = signal_collection / data_max

In [8]:
signal_collection.min(), signal_collection.max()

(-1.9539306399697487, 2.2628056343165652)

# 1. Make clearer disctinction between training, test, validation data!
To be sure the network is able to make predictions using unseen data, the dataset could be split according to persons! Unfortunately we only have data here for 57 persons (24 in group 1 and 33 in group 2).  
This makes this approach a bit complicated

## Idea for next steps:
We could also loop through different test-person/train-person splits and independently train models on those datasets. Then, in the the end the outcome would be averaged over all of those. This way we would make better use of the data we have!

In [9]:
groups = []
for ID in range(len(metadata)):
    if ID == 0:
        low = 0
    else:
        low = int(metadata[ID-1][2])
        
    groups.append(int(label_collection[low]/3))

In [10]:
group_1_ids = np.where(np.array(groups) == 1)[0]
group_2_ids = np.where(np.array(groups) == 1)[0]

In [11]:
# inspect group 2 ID's
group_2_ids

array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43], dtype=int64)

In [12]:
def create_averaged_data(signal_collection, 
                         label_collection, 
                         metadata, 
                         n_epochs, 
                         selected_ids):
    """ 
    Function to create averages of n_epochs epochs.
    
    """
    # create new data collection:
    X_data = np.zeros((0, signal_collection.shape[1], signal_collection.shape[2]))
    y_data = []
    
    for ID in selected_ids:
        print("person ID:", ID, ". Originally from file: ", metadata[ID][1])
        if ID == 0:
            low = 0
        else:
            low = int(metadata[ID-1][2])
        high = int(metadata[ID][2])

        # Create class=3 averages (n_class3_per_patient times)
        for i in range(n_class3_per_patient):      
            select_label = np.where(label_collection[low:high] == 3)[0]
            if len(select_label) == 0:
                select_label = np.where(label_collection[low:high] == 6)[0]
                group = 2
            else:
                group = 1
            if len(select_label) >= n_epochs:        
                select = np.random.choice(select_label, n_epochs, replace=False)
            else:
                print("Found only", len(select_label), " epochs and will take those!")
                signal_averaged = np.mean(signal_collection[select_label,:,:], axis=0)
                break
            signal_averaged = np.mean(signal_collection[select,:,:], axis=0)
            X_data = np.concatenate([X_data, np.expand_dims(signal_averaged, axis=0)], axis=0)
            y_data.append(group*3)

         # Create class=13 averages (n_class13_per_patient times)
        for i in range(n_class13_per_patient):      
            select_label = np.where(label_collection[low:high] == 13)[0]
            if len(select_label) == 0:
                select_label = np.where(label_collection[low:high] == 26)[0]
                group = 2
            else:
                group = 1
            if len(select_label) >= n_epochs:        
                select = np.random.choice(select_label, n_epochs, replace=False)
            else:
                print("Found only", len(select_label), " epochs and will take those!")
                signal_averaged = np.mean(signal_collection[select_label,:,:], axis=0)
                break
            signal_averaged = np.mean(signal_collection[select,:,:], axis=0)
            X_data = np.concatenate([X_data, np.expand_dims(signal_averaged, axis=0)], axis=0)
            y_data.append(group*13)

        # Create class=66 averages (n_class66_per_patient times)
        for i in range(n_class66_per_patient):      
            select_label = np.where(label_collection[low:high] == 66)[0]
            if len(select_label) == 0:
                select_label = np.where(label_collection[low:high] == 132)[0]
                group = 2
            else:
                group = 1
            if len(select_label) >= n_epochs:        
                select = np.random.choice(select_label, n_epochs, replace=False)
            else:
                print("Found only", len(select_label), " epochs and will take those!")
                signal_averaged = np.mean(signal_collection[select_label,:,:], axis=0)
                break
            signal_averaged = np.mean(signal_collection[select,:,:], axis=0)
            X_data = np.concatenate([X_data, np.expand_dims(signal_averaged, axis=0)], axis=0)
            y_data.append(group*66)

    return X_data, y_data

In [13]:
n_epochs = 30 # Average over n_epochs epochs
n_class3_per_patient = 20
n_class13_per_patient = 10
n_class66_per_patient = 10

# Make selection 
group_1_ids = np.where(np.array(groups) == 1)[0]
group_2_ids = np.where(np.array(groups) == 2)[0]
keep_group1_for_test = 10
keep_group2_for_test = 10

# Initialize random numbers to get reproducible results 
np.random.seed(42)

# Make random selection
selected_ids_test = [x for x in np.concatenate([np.random.choice(group_1_ids, keep_group1_for_test),
                                   np.random.choice(group_2_ids, keep_group2_for_test)])]

selected_ids = [x for x in range(len(metadata)) if not x in selected_ids_test]

In [14]:
X_data, y_data = create_averaged_data(signal_collection, 
                         label_collection, 
                         metadata, 
                         n_epochs, 
                         selected_ids)

person ID: 0 . Originally from file:  034_17_mc_mmn36_wk.cnt
person ID: 1 . Originally from file:  036_17_mc_mmn36_wk.cnt
person ID: 2 . Originally from file:  175_17_jd_mmn_wk.cnt
person ID: 3 . Originally from file:  305_17_jc_mmn36_wk.cnt
person ID: 4 . Originally from file:  306_17_mc_mmn36_wk.cnt
person ID: 5 . Originally from file:  307_17_jc_mmn36_wakker.cnt
person ID: 6 . Originally from file:  308_17_jc_mmn36_wk.cnt
person ID: 9 . Originally from file:  337_17_jc_mmn36_wk.cnt
person ID: 10 . Originally from file:  343_17_mc_mmm36_wk.cnt
person ID: 14 . Originally from file:  420_17_md_mmn.cnt
person ID: 15 . Originally from file:  428_17_md_mmn.cnt
person ID: 16 . Originally from file:  434_17_jd_mmn.cnt
person ID: 17 . Originally from file:  435_17_md_mmn36_wk.cnt
person ID: 18 . Originally from file:  436_17_jd_mmn36_wk.cnt
person ID: 19 . Originally from file:  443_17_jd_mmn36_wk.cnt
person ID: 20 . Originally from file:  454_17_md_mmn36_wk.cnt
Found only 27  epochs and wil

In [15]:
X_test, y_test = create_averaged_data(signal_collection, 
                         label_collection, 
                         metadata, 
                         n_epochs, 
                         selected_ids_test)

person ID: 7 . Originally from file:  309_17_jc_mmn.cnt
person ID: 39 . Originally from file:  637-479-17m-mc-mmn36.cnt
person ID: 34 . Originally from file:  609-158-17m-jc-mmn36.cnt
person ID: 11 . Originally from file:  345_17_mc_mmn36_wk.cnt
person ID: 8 . Originally from file:  314_17_mc_mmn36_wk.cnt
person ID: 40 . Originally from file:  639-484-17m-jc-mmn36.cnt
person ID: 7 . Originally from file:  309_17_jc_mmn.cnt
person ID: 38 . Originally from file:  636-468-17m-jc-mmn36.cnt
person ID: 42 . Originally from file:  642-485-17m-jc-mmn36.cnt
person ID: 11 . Originally from file:  345_17_mc_mmn36_wk.cnt
person ID: 21 . Originally from file:  455_17_jd_mmn36_wk.cnt
person ID: 47 . Originally from file:  724-116-17m-jr-mmn36.cnt
person ID: 47 . Originally from file:  724-116-17m-jr-mmn36.cnt
person ID: 13 . Originally from file:  413_17_jd_mmn_2.cnt
person ID: 45 . Originally from file:  719-079-jr-17m-mmn36.cnt
person ID: 12 . Originally from file:  406_17_md_mmn.cnt
person ID: 47

In [16]:
print(X_data.shape, len(y_data))
print(X_test.shape, len(y_test))
#print(y_data[:100])

(1650, 30, 501) 1650
(800, 30, 501) 800


In [17]:
Xmean = np.concatenate([X_data, X_test]).mean()
X_data = X_data - Xmean
X_test = X_test - Xmean

Xmax = np.concatenate([X_data, X_test]).max()

X_data = X_data / Xmax
X_test = X_test / Xmax

In [18]:
X_data.mean(), X_data.max(),X_data.min()

(-0.002006264229741046, 0.9934850700415665, -1.13929890512535)

## Split training data
### Now the test dataset is already entirely seperate from the rest! 
+ validation data, used to monitor the model progress and avoid overfitting.
+ testing data, meant for final check on model performance.
+ --> Create validation and test data set from seperated data!!

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
X_train, y_train = shuffle(X_data, y_data, random_state=1)

In [20]:
print('Train set size:', X_train.shape[0])
print('Validation set size:', X_val.shape[0])
print('Test set size:', X_test.shape[0])
print()
print("X_train mean, min, max: ", np.mean(X_train), np.min(X_train), np.max(X_train))

Train set size: 1650
Validation set size: 400
Test set size: 400

X_train mean, min, max:  -0.0020062642297410485 -1.13929890512535 0.9934850700415665


## Switch to 1-hot encoding for labels
+ We have six categories or classes. Those are best represented by a so called 1-hot encoding. This means nothing else than simply a binary 0-or-1 for every class.. 

In [21]:
from sklearn.preprocessing import LabelBinarizer
label_transform = LabelBinarizer()

y_train_binary = label_transform.fit_transform(np.array(y_train).astype(int))
y_val_binary = label_transform.fit_transform(np.array(y_val).astype(int))
y_test_binary = label_transform.fit_transform(np.array(y_test).astype(int))

In [22]:
y_val_binary[:10,:]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [23]:
# Show found labels:
label_transform.classes_

array([  3,   6,  13,  26,  66, 132])

Check distribution accross the 6 label categories:

In [24]:
labels = list(label_transform.classes_)
frequencies = y_train_binary.mean(axis=0)
frequencies_df = pd.DataFrame(frequencies, index=labels, columns=['frequency'])
frequencies_df

,frequency
3,0.193939
6,0.315152
13,0.090909
26,0.157576
66,0.090909
132,0.151515


### Note:
We have more data on group 2 than on group 1. And far more data for stimuli 3 than for stimuli 13 and 66 (not surprising). 

--> post on balancing datasets: https://towardsdatascience.com/handling-imbalanced-datasets-in-deep-learning-f48407a0e758

### Needs some thinking on how to balance the data set !
e.g. by frequency dependend selection rule, or by defining a suitied special loss function....

In [25]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train)
                                               ,y_train)

In [26]:
class_weight

array([0.859375  , 0.52884615, 1.83333333, 1.05769231, 1.83333333,
       1.1       ])

In [27]:
class_weight = {0: class_weight[0],
               1: class_weight[1],
               2: class_weight[2],
               3: class_weight[3],
               4: class_weight[4],
               5: class_weight[5]}

## Define model architecture

In [66]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [28]:
import tensorflow as tf
from tensorflow.keras import layers

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
output_file = 'CNN_EEG_classifier_avg_02'

checkpointer = ModelCheckpoint(filepath = PATH_MODELS + output_file + ".hdf5", monitor='val_acc', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_acc', patience=5, verbose=1)

In [35]:
# Simple CNN model
n_timesteps = 501
n_features = 30
n_outputs = 6

model = tf.keras.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=20, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(layers.AveragePooling1D(pool_size=2))
#model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Conv1D(filters=64, kernel_size=10, activation='relu'))
model.add(layers.AveragePooling1D(pool_size=2))
#model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(layers.AveragePooling1D(pool_size=2))
#model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(layers.AveragePooling1D(pool_size=2))
#model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 482, 32)           19232     
_________________________________________________________________
average_pooling1d_8 (Average (None, 241, 32)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 232, 64)           20544     
_________________________________________________________________
average_pooling1d_9 (Average (None, 116, 64)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 112, 128)          41088     
_________________________________________________________________
average_pooling1d_10 (Averag (None, 56, 128)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 54, 128)           49280     
__________

In [37]:
epochs = 50
batch_size = 32

# fit network
model.fit(np.swapaxes(X_train,1,2), 
          y_train_binary, 
          validation_data=(np.swapaxes(X_val,1,2), y_val_binary), 
          epochs=epochs, 
          batch_size=batch_size,
          class_weight = class_weight,
          callbacks = [checkpointer, earlystopper])

Train on 1650 samples, validate on 400 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1632/1650 [============================>.] - ETA: 0s - loss: 1.9217 - acc: 0.2102
Epoch 00001: val_acc improved from -inf to 0.20750, saving model to C:\OneDrive - Netherlands eScience Center\Project_ePodium\trained_models\CNN_EEG_classifier_avg_02.hdf5
1650/1650 [==============================] - 18s 11ms/sample - loss: 1.9195 - acc: 0.2097 - val_loss: 1.6885 - val_acc: 0.2075
Epoch 2/50
1632/1650 [============================>.] - ETA: 0s - loss: 1.6247 - acc: 0.3039
Epoch 00002: val_acc improved from 0.20750 to 0.35750, saving model to C:\OneDrive - Netherlands eScience Center\Project_ePodium\trained_models\CNN_EEG_classifier_avg_02.hdf5
1650/1650 [==============================] - 16s 10ms/sample - loss: 1.6267 - acc: 0.3030 - val_loss: 1.4658 - val_acc: 0.3575
Epoch 3/50
1632/1650 [============================>.] - ETA: 0s - loss: 1.4830 - acc: 0.3578
Epoch 00003: val_acc did 

## Seems to overfit on the training data and not be able to predict well the validation data

In [38]:
# Evaluate the model
_, train_acc = model.evaluate(np.swapaxes(X_train,1,2), y_train_binary, verbose=0)
_, test_acc = model.evaluate(np.swapaxes(X_test,1,2), y_test_binary, verbose=0)

In [39]:
print("Accuracy on train dataset:", train_acc)
print("Accuracy on test dataset:", test_acc)

Accuracy on train dataset: 0.5478788
Accuracy on test dataset: 0.3925


In [40]:
Xtest = np.swapaxes(X_test,1,2)

# Check model predictions:
y_pred_proba = model.predict_proba(Xtest)
y_pred_classes = model.predict_classes(Xtest)

In [41]:
y_test_05 = np.array(y_test.copy())
y_test_05[y_test_05 == 132] = 5
y_test_05[y_test_05 == 66] = 4
y_test_05[y_test_05 == 26] = 3
y_test_05[y_test_05 == 13] = 2
y_test_05[y_test_05 == 6] = 1
y_test_05[y_test_05 == 3] = 0

print(y_test_05[:20].astype(int))
print(y_pred_classes[:20])

[0 5 0 1 0 1 0 5 0 1 0 4 0 2 0 1 5 1 2 0]
[0 5 0 0 0 0 4 5 0 0 0 4 0 5 2 0 5 0 4 3]


In [43]:
# Calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test_05, y_pred_classes))

0.375


## Check if groups are predicted correctly:

In [48]:
y_test_12 = np.array(y_test.copy())
y_test_12[y_test_12 == 132] = 2
y_test_12[y_test_12 == 66] = 1
y_test_12[y_test_12 == 26] = 2
y_test_12[y_test_12 == 13] = 1
y_test_12[y_test_12 == 6] = 2
y_test_12[y_test_12 == 3] = 1

y_pred_12 = y_pred_classes.copy()
y_pred_12[y_pred_12 == 5] = 2
y_pred_12[y_pred_12 == 4] = 1
y_pred_12[y_pred_12 == 3] = 2
y_pred_12[y_pred_12 == 2] = 1
y_pred_12[y_pred_12 == 1] = 2
y_pred_12[y_pred_12 == 0] = 1

print(y_test_12[:30].astype(int))
print(y_pred_12[:30])

[1 2 1 2 1 2 2 2 1 2 1 1 1 1 2 2 2 2 1 1 1 2 1 1 2 1 2 1 2 1]
[1 2 1 1 1 1 2 2 1 1 1 2 1 2 2 1 2 1 2 2 2 1 2 2 2 1 2 2 2 2]


In [49]:
np.sum(y_test_12 == y_pred_12)/ y_pred_12.shape[0]

0.465

## Observation:
So far this is not working as a proper discriminator between group1 and group2!  
The test dataset contains about equal amounts of data from both groups, and the model does do better than random guessing (50% hits).

In [44]:
# Confusion matrix:
M_confusion = metrics.confusion_matrix(y_test_05, y_pred_classes)
M_confusion

array([[71, 19, 33, 11, 10,  3],
       [87, 12,  0,  1,  1,  0],
       [ 3,  3, 20, 17,  4,  5],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  2, 16, 25],
       [ 1,  0,  8,  6,  6, 31]], dtype=int64)

In [47]:
np.sum(y_test_05 == 0), np.sum(y_pred_classes == 0)

(147, 162)

### Interestingly, it might be that the model at least does see a difference between the types of stimuli!

In [50]:
y_test_bak_dak = np.array(y_test.copy())
y_test_bak_dak[y_test_bak_dak == 132] = 2
y_test_bak_dak[y_test_bak_dak == 66] = 2
y_test_bak_dak[y_test_bak_dak == 26] = 2
y_test_bak_dak[y_test_bak_dak == 13] = 2
y_test_bak_dak[y_test_bak_dak == 6] = 1
y_test_bak_dak[y_test_bak_dak == 3] = 1

y_pred_bak_dak = y_pred_classes.copy()
y_pred_bak_dak[y_pred_bak_dak == 5] = 2
y_pred_bak_dak[y_pred_bak_dak == 4] = 2
y_pred_bak_dak[y_pred_bak_dak == 3] = 2
y_pred_bak_dak[y_pred_bak_dak == 2] = 2
y_pred_bak_dak[y_pred_bak_dak == 1] = 1
y_pred_bak_dak[y_pred_bak_dak == 0] = 1

print(y_test_bak_dak[:30].astype(int))
print(y_pred_bak_dak[:30])

[1 2 1 1 1 1 2 2 1 1 1 2 1 2 2 1 2 1 2 1 2 1 2 1 2 1 2 2 2 2]
[1 2 1 1 1 1 2 2 1 1 1 2 1 2 2 1 2 1 2 2 2 1 2 1 2 1 2 2 2 2]


In [51]:
np.sum(y_test_bak_dak == y_pred_bak_dak)/ y_pred_bak_dak.shape[0]

0.9525

## Interestingly though,...
That would mean that the model is correct in 96% of all cases in distinguishing 'bak' from 'dak'.  
--> **Careful: Needs to be checked if my assumptions about the stimuli are correct...**